# Valuing European-Style Swaptions Across Models

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.libor import *
from financepy.finutils import *
from financepy.market.curves.FinLiborCurve import FinLiborCurve
from financepy.market.curves.FinInterpolate import FinInterpMethods

###################################################################
#   WELCOME TO FINANCEPY BETA VERSION 0.168 - UNDER DEVELOPMENT   #
# FinancePy it is distributed FREE & WITHOUT ANY WARRANTY AND     #
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. For more   #
# details see the repo at https://github.com/domokane/FinancePy   #
###################################################################



## Building a Libor Discount Curve

In [3]:
valuationDate = FinDate(28, 2, 2014)
settlementDate = FinDate(4, 3, 2014)

We have a vector of dates and zero rates

In [4]:
depoDCCType = FinDayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = FinLiborDeposit(settlementDate, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
fixedDCCType = FinDayCountTypes.ACT_365_ISDA
fixedFreqType = FinFrequencyTypes.SEMI_ANNUAL

swap = FinLiborSwap(settlementDate, "3Y", 0.00790, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "4Y", 0.01200, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "5Y", 0.01570, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "6Y", 0.01865, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "7Y", 0.02160, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "8Y", 0.02350, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "9Y", 0.02540, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "10Y", 0.0273, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "15Y", 0.0297, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "20Y", 0.0316, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "25Y", 0.0335, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "30Y", 0.0354, fixedFreqType, fixedDCCType); swaps.append(swap)

liborCurve = FinLiborCurve("USD_LIBOR", valuationDate, depos, [], swaps, FinInterpMethods.LINEAR_ZERO_RATES)

In [5]:
exerciseDate = settlementDate.addTenor("5Y")
swapMaturityDate = exerciseDate.addTenor("5Y")
swapFixedCoupon = 0.040852
swapFixedFrequencyType = FinFrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = FinDayCountTypes.THIRTY_E_360_ISDA
swapFloatFrequencyType = FinFrequencyTypes.QUARTERLY
swapFloatDayCountType = FinDayCountTypes.ACT_360
swapNotional = ONE_MILLION
swaptionType = FinLiborSwaptionTypes.PAYER  
calendarType = FinCalendarTypes.WEEKEND
busDayAdjustType = FinBusDayAdjustTypes.NONE
dateGenRuleType = FinDateGenRuleTypes.BACKWARD

In [6]:
swaption = FinLiborSwaption(settlementDate,
                            exerciseDate,
                            swapMaturityDate,
                            swaptionType,
                            swapFixedCoupon,
                            swapFixedFrequencyType,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            calendarType, 
                            busDayAdjustType,
                            dateGenRuleType)

In [7]:
print(swaption)

SETTLEMENT DATE: TUE 4 MAR 2014
EXERCISE DATE: MON 4 MAR 2019
SWAPTION TYPE: FinLiborSwaptionTypes.PAYER
MATURITY DATE: MON 4 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FinFrequencyTypes.QUARTERLY
FLOAT DAY COUNT: FinDayCountTypes.ACT_360



## Valuation using Black's Model

In [8]:
model = FinModelBlack(0.1533)

In [9]:
swaption.value(settlementDate, liborCurve, model)

23151.19239437166

## Valuation using Shifted Black

In [10]:
model = FinModelBlackShifted(0.1533, -0.008)

In [11]:
swaption.value(settlementDate, liborCurve, model)

27681.37058274581

##  Valuation using SABR

In [12]:
model = FinModelSABR(0.132, 0.5, 0.5, 0.5)

In [13]:
swaption.value(settlementDate, liborCurve, model)

104535.337040495

## Valuation using Shifted SABR

In [44]:
model = FinModelSABRShifted(0.1, 0.2, 0.1, 0.15, -0.005)

In [45]:
swaption.value(settlementDate, liborCurve, model)

163409.8258927529

## Valuation using Hull-White

In [34]:
model1 = FinModelRatesHW(0.01, 0.02)

In [35]:
swaption.value(settlementDate, liborCurve, model1)

34158.87046477075

## Internals

In [17]:
print(swaption)

EXERCISE DATE: MON 4 MAR 2019
SWAPTION TYPE: FinLiborSwaptionTypes.PAYER
MATURITY DATE: MON 4 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FinFrequencyTypes.QUARTERLY
FLOAT DAY COUNT: FinDayCountTypes.ACT_360
PV01: 4.161289611071258
FWD SWAP RATE: 4.085954343978203
FWD DF TO EXPIRY: 0.9237073134954512


We can see that the forward swap rate almost equals the fixed coupon. The underlying swap is close to being ATM forward.

In [47]:
swaption.printSwapFixedLeg()

START DATE: MON 4 MAR 2019
MATURITY DATE: MON 4 MAR 2024
COUPON (%): 4.0852
FIXED LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE TUE 4 MAR 2014
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
 WED 4 SEP 2019  0.5000000     20426.00   0.90860187     18559.10     18559.10
 WED 4 MAR 2020  0.5000000     20426.00   0.89253745     18230.97     36790.07
 FRI 4 SEP 2020  0.5000000     20426.00   0.87505999     17873.98     54664.05
 THU 4 MAR 2021  0.5000000     20426.00   0.85687536     17502.54     72166.58
 SAT 4 SEP 2021  0.5000000     20426.00   0.84093396     17176.92     89343.50
 FRI 4 MAR 2022  0.5000000     20426.00   0.82471280     16845.58    106189.08
 SUN 4 SEP 2022  0.5000000     20426.00   0.80761260     16496.29    122685.38
 SAT 4 MAR 2023  0.5000000     20426.00   0.79033077     16143.30    138828.68
 MON 4 SEP 2023  0.5000000     20426.00   0.77210016     15770.92    154599.

Copyright (c) 2020 Dominic O'Kane